In [2]:
import sys
sys.path.append("..")

In [4]:
from mango.mango import Mango
from mango.concepts import IdentityConcept
from mango.actions import FullCompatibility
from mango.environments import DummyEnvironment
from gymnasium import spaces

n_layers = 2

mango = Mango(
    environment=DummyEnvironment(),
    concepts=[IdentityConcept() for i in range(n_layers)],
    action_compatibilities=[FullCompatibility(spaces.Discrete(i+3)) for i in range(n_layers)],
    base_concept=IdentityConcept(),
)

mango

c:\Users\dsart\anaconda3\envs\python310base\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Mango(
  (0): MangoEnv(
    (concept): IdentityConcept()
    (environment): DummyEnvironment(state=0, observation_space=Discrete(2), action_space=Discrete(2))
  )
  (1): MangoLayer(
    (concept): IdentityConcept()
    (act_comp): FullCompatibility(action_space=Discrete(3))
    (policy): DQnetPolicyMapper(comand_space=Discrete(3), action_space=Discrete(2))
  )
  (2): MangoLayer(
    (concept): IdentityConcept()
    (act_comp): FullCompatibility(action_space=Discrete(4))
    (policy): DQnetPolicyMapper(comand_space=Discrete(4), action_space=Discrete(3))
  )
)